In [4]:
type Command =
  | CDRoot
  | CD of name: string
  | CDParent
  | LSFile of name: string * size: int

let parseCommand (s: string) = 
  match s.Split(' ') with
    | [|"$"; "cd"; "/" |] -> Some CDRoot
    | [|"$"; "cd"; ".." |] -> Some CDParent
    | [|"$"; "cd"; dirName |] -> Some (CD dirName)
    | [|"$"; "ls"|] | [|"dir"; _|] -> None // We don't need these lines
    | [|size; fileName|] -> Some (LSFile (fileName, int size))
    | _ -> invalidArg s "Invalid argument"

let commands = 
  File.ReadAllLines "inputs/07/input.txt"
  |> Array.toList
  |> List.choose parseCommand

In [5]:
type FSItem =
  | FSDir of FSDir
  | FSFile of FSFile
  member self.size = FSItem.sizeOf self
  static member sizeOf = function FSDir {size = size} | FSFile {size = size} -> size
  static member isFile = function FSDir _ -> false | FSFile _ -> true
  static member isDir = not << FSItem.isFile
and FSDir = {name: string; size: int; children: FSItem list} with 
  static member Default = {name = ""; size = 0; children = []}
  member self.addChild child = 
    {
      self with 
        children = child :: self.children;
        size = child.size + self.size
    }
and FSFile = {name: string; size: int}

let parseFilesystem = function
  | CDRoot :: commands -> 
    let rec parseDir commands (currentDir: FSDir) =
      match commands with
      | CD dirName :: rest -> 
        let child, cmds = {FSDir.Default with name = dirName} |> parseDir rest 
        currentDir.addChild (FSDir child)
        |> parseDir cmds

      | LSFile (fileName, size) :: rest -> 
        currentDir.addChild (FSFile {name = fileName; size = size})
        |> parseDir rest

      | CDParent :: rest -> currentDir, rest
      | [] -> currentDir, []
      | CDRoot :: rest -> invalidArg "cd /" "Only one root expected"
    parseDir commands {FSDir.Default with name = "/"} |> fst
  | _ -> invalidArg "commands" "Expected 'cd /' to begin command list"

let filesystem = parseFilesystem commands

let rec traverse dir = seq {
    yield FSDir dir
    for child in dir.children do
      match child with
      | FSFile _ -> yield child
      | FSDir childDir -> yield! traverse childDir
  }

traverse filesystem
|> Seq.filter FSItem.isDir
|> Seq.map FSItem.sizeOf
|> Seq.filter ((>=) 100000)
|> Seq.sum

1315285

In [6]:
let diskSpace = 70_000_000
let neededFreeSpace = 30_000_000
let usedSpace = filesystem.size
let freeSpace = diskSpace - usedSpace
let needToFree = neededFreeSpace - freeSpace

traverse filesystem
|> Seq.filter FSItem.isDir
|> Seq.map FSItem.sizeOf
|> Seq.sort
|> Seq.find ((<=) needToFree)

9847279